In [1]:
import pandas as pd
import numpy as np
tourney_df = pd.read_csv('data/TourneyDetailedResults.csv')
teams_df = pd.read_csv('data/Teams.csv')
reg_season_df = pd.read_csv('data/RegularSeasonCompactResults.csv')
team_dict = dict(zip(teams_df['Team_Id'].values, teams_df['Team_Name'].values))
tourney_df['Wteam_name'] = tourney_df['Wteam'].map(team_dict)
tourney_df['Lteam_name'] = tourney_df['Lteam'].map(team_dict)

In [2]:
Ngames = tourney_df.shape[0]
tourney_df['Flip'] = np.random.randint(2,size=Ngames)
seasons = tourney_df['Season'].unique()
in_reg_season = dict(zip(seasons, [reg_season_df['Season'] == i for i in seasons]))
valid = reg_season_df['Wteam'] != 0

In [3]:
def team_name0(x):
    if x['Flip']:
        return x['Lteam_name']
    else:
        return x['Wteam_name']

def team_name1(x):
    if x['Flip']:
        return x['Wteam_name']
    else:
        return x['Lteam_name']
    
def wins0(x):  
    wins = reg_season_df.loc[valid & in_reg_season[x['Season']], 'Wteam'].value_counts()  
    if x['Flip']:
        return wins[x['Lteam']]
    else:
        return wins[x['Wteam']]
    
def wins1(x):  
    wins = reg_season_df.loc[valid & in_reg_season[x['Season']], 'Wteam'].value_counts()  
    if x['Flip']:
        return wins[x['Wteam']]
    else:
        return wins[x['Lteam']]

In [4]:
fList = [team_name0, team_name1, wins0, wins1]

In [5]:
for feature in fList:
    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)

tourney_df['Winner'] = tourney_df['Flip']

In [6]:
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lblk,Lpf,Wteam_name,Lteam_name,Flip,team_name0,team_name1,wins0,wins1,Winner
0,2003,134,1421,92,1411,84,N,1,32,69,...,0,22,UNC Asheville,TX Southern,1,TX Southern,UNC Asheville,18,13,1
1,2003,136,1112,80,1436,51,N,0,31,66,...,3,15,Arizona,Vermont,0,Arizona,Vermont,25,19,0
2,2003,136,1113,84,1272,71,N,0,31,59,...,5,18,Arizona St,Memphis,0,Arizona St,Memphis,18,23,0
3,2003,136,1141,79,1166,73,N,0,29,53,...,6,21,C Michigan,Creighton,0,C Michigan,Creighton,23,29,0
4,2003,136,1143,76,1301,74,N,1,27,64,...,8,19,California,NC State,1,NC State,California,18,21,1


In [6]:
Ngames = tourney_df.shape[0]
y = tourney_df['Flip'].values
X = np.asarray([features(tourney_df.loc[i, 'Wteam'], tourney_df.loc[i, 'Lteam'], tourney_df.loc[i, 'Season'], y[i]) for i in range(Ngames)])

In [7]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [8]:
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
from sklearn import metrics
y_pred = model.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.625368731563


In [11]:
print(y[0:100])

[0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0
 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1
 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 0 1 0]


In [18]:
print(X[0:10])

[[13 18]
 [25 19]
 [23 18]
 [23 29]
 [21 18]
 [21 23]
 [17 24]
 [23 17]
 [24 23]
 [23 22]]
